In [1]:
# Load packages
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import inception_v3, Inception_V3_Weights
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder
from sklearn.impute import SimpleImputer
from pathlib import Path
import shared_functions as sf

In [2]:
# Define model & file name
model_name = 'IMGModel_16'
file_name = 'property-sales_new-york-city_2022_image-classes'

In [3]:
# Create output directory for exports
Path(f'../models/{model_name}').mkdir(parents=True, exist_ok=True)

In [4]:
# Load subset keys as list
subset_keys = pd.read_csv(f'../data/processed/subset_keys_image_class.csv').squeeze().to_list()

In [5]:
# Load subset index as series
subset_index = pd.read_csv(f'../data/processed/subset_index_image_class.csv', index_col=0)

In [6]:
# Use GPU when possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print(f'Device type: {device.upper()}')

Device type: CPU


In [7]:
# Set random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [8]:
dataset_params = {
    'data': f'../data/processed/{file_name}.parquet',
    'target_name': 'price_class',
    'image_directory': '../data/raw/satellite-images_new-york-city_2022_640x640_16/',
    'image_transformation': transforms.Compose([
        transforms.CenterCrop((600, 600)), # crop image borders by margin of 20px to remove text from 640x640
        transforms.Resize((299, 299)), # resize image to 299x299
        transforms.ToTensor(),  # convert image to PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # noarmlize based on ImageNet data
        ]),
    'subset_index': '../data/processed/subset_index_image_class.csv'
    }

In [9]:
# Instantiate datasets
subsets = {subset_key: sf.MultiModalDataset(**dataset_params, subset=subset_key) for subset_key in subset_keys}
dataset = sf.MultiModalDataset(**dataset_params)

In [10]:
# Define model architecture
class IMGModel_16(nn.Module):
    # Define model components
    def __init__(self):
        super().__init__()
        
        # Define image model
        self.ImageModel = inception_v3(weights=Inception_V3_Weights.DEFAULT)
        self.ImageModel.aux_logits = False
        for parameter in self.ImageModel.parameters():
            parameter.requires_grad = False
        self.ImageModel.fc = nn.Linear(self.ImageModel.fc.in_features, 2)
    
    # Define forward pass
    def forward(self, X_text, X_image):
        y = self.ImageModel(X_image)
        return y

In [11]:
# Instantiate model
model = IMGModel_16().to(device)

In [12]:
# Calculate number of model parameters
n_params = sum(parameter.numel() for parameter in model.parameters())
print(f'# model paramters: {n_params}')

# model paramters: 25116362


In [13]:
# Do not train if already trained
if Path(f'../models/{model_name}/state_dict.pt').is_file() and Path(f'../models/{model_name}/history.csv').is_file():
    # Load optimal weights and history
    model.load_state_dict(torch.load(f'../models/{model_name}/state_dict.pt', map_location='cpu'))
    history = pd.read_csv(f'../models/{model_name}/history.csv', index_col=0)
    print('Skipping training and loading optimal weights from previous training!')
else:
    # Train model
    model, history = sf.train_image_model(
        model=model,
        dataset_train=subsets['train'],
        dataset_val=subsets['val'],

        # Define loss & optimizer
        loss_function=nn.CrossEntropyLoss().to(device),
        optimizer=optim.SGD(model.parameters(), lr=0.01, momentum=0.9),

        # Define computing device
        device=device,

        # Define training parameters
        epochs=25,
        patience=25,
        delta=0,
        batch_size=1024,
        shuffle=True,
        num_workers=8,
        pin_memory=True,

        # Define save locations
        save_state_dict_as=f'../models/{model_name}/state_dict.pt',
        save_history_as=f'../models/{model_name}/history.csv'
        )

  0%|          | 0/17 [00:43<?, ?it/s]
